In [1]:
(require '[clojupyter.misc.helper :as helper])
(run! helper/add-dependencies '[[sicmutils "0.16.0"]])

(ns chp3
    (:refer-clojure :exclude [partial zero? + - * / ref compare])
    (:require [sicmutils.env :refer :all]
              [sicmutils.value :as v]
              [sicmutils.calculus.manifold :refer [Rn, R2, make-patch, make-manifold
                                                   coordinate-system-at]]))
(def render (comp ->infix simplify))

#'chp3/render

## 3.1 Vector Fields

For a function $f$ on $R^n$, 
$$ f(x + \Delta x) \approx f(x) + (D f(x)) \Delta x$$

- the derivative $Df(x)$ is independent of $\Delta x$, but depends on the coordinates
- The product $(D f(x)) \Delta x$ is invariant under change of coordinates. (directional derivative)
- $(D f(x)) \Delta x$ is the directional derivative of $f$ at $x$ w.r.t. the vector specified by the tuple of components $\Delta x$ in the coordinate system. (This is at a single point)
- generalize this idea to assign a vector ($\Delta x$) to every point, making a __vector field__
- let $b$ be a map from coordinates ($x$) to vectors ($\Delta x$), we have a directional derivative of $f$ at each point $x$ determined by $b$
$$ D_b (f) (x) = (Df(x)) b(x) $$

Now we bring this idea back to the manifold. 
- A __vector field on a manifold__ is an assignment of a vector to each point on the manifold. 
- In differential geometry, a __vector__  is an operator that maps a manifold function, to directional derivatives at each point. 

Notation explained
- $\mathsf{m}$: a point on a manifold
- $\mathsf{v}$: a vector field on the manifold
- $\mathsf{f}$: a real-valued function on the manifold. (a manifold function)
- $\mathsf{v}(\mathsf{f})$: directional derivative of the function $\mathsf{f}$ determined by $\mathsf{v}$, which is a manifold function
- $\mathsf{v}(\mathsf{f})(\mathsf{m})$: directional derivative of the function $\mathsf{f}$ determined by $\mathsf{v}$ at point $\mathsf{m}$. A scalar number. 

In [2]:
(clojure.repl/source components->vector-field)

(defn components->vector-field
  [components coordinate-system & [name]]
  (let [name (or name `(~'vector-field ~components))]
    (procedure->vector-field (vector-field-procedure components coordinate-system) name)))


nil

In [3]:
;; a signiture for functions that map an up structure of two reals to a real.
(def R2->R '(-> (UP Real Real) Real))

#'chp3/R2->R

In [4]:
;; define a manifold point using its rectangular coordinates
(def R2-rect-point ((point R2-rect) (up 'x0 'y0)))

#'chp3/R2-rect-point

In [5]:
;; a vector field v with coordinate component functions named b0 and b1
;; coordinate component functions are also called coefficient functions
(def v
    (components->vector-field
     (up (literal-function 'b0 R2->R)   ;; coefficient function
         (literal-function 'b1 R2->R))  ;; coefficient function
     R2-rect))

#'chp3/v

In [6]:
;; a literal vector field
(def v2
    (literal-vector-field 'b R2-rect))

#'chp3/v2

A useful way to characterize a vector field in a particular coordinate system is by applying it to coordinate functions, so that we get the following _coordinate component functions_ or _coefficient functions_ of the vector field.
$$ b_{\chi, \mathsf{v}}^{i} = \mathsf{v}(\chi^i) \circ \chi^{-1} $$
- coordinate components are function of the coordinate tuple, not of a manifold point
- the derectional derivatives can be written in terms of coordinate components

Notation explained
- $\mathsf{f} \circ \chi^{-1}$: the coordinate representation of the manifold function $\mathsf{f}$.
- $D(\mathsf{f} \circ \chi^{-1})$: derivative of the manifold function in a coordinate frome. 
- $D(\mathsf{f} \circ \chi^{-1}) b_{\chi, \mathsf{v}}$: weight the components of derivative of the function with the coordinate components of the vector field and sum up. This gives the directional derivative of a coordinate point!

$$\mathsf{v}(\mathsf{f})(\mathsf{m}) 
= D(\mathsf{f} \circ \chi^{-1}) (\chi(\mathsf{m})) 
  b_{\chi, \mathsf{v}} (\chi(\mathsf{m}))$$
$$\mathsf{v}(\mathsf{f})(\mathsf{m}) 
= \sum_i \partial_i (\mathsf{f} \circ \chi^{-1}) (\chi(\mathsf{m})) 
  b_{\chi, \mathsf{v}}^i (\chi(\mathsf{m}))$$
  
- The coordinate tuple $b_{\chi, \mathsf{v}} (x)$ is an up structure.
- For any vector field $\mathsf{v}$ the coefficients $b_{\chi, \mathsf{v}} (x)$ are different for different coordinate functions $\chi$
- we'll drop the subscripts on $b$ for brevity

In [20]:
;; apply the vector field (as an operator) to an arbitrary manifold function
;; v(f)(m)
((v (literal-manifold-function 'f-rect R2-rect)) R2-rect-point)

(+ (* (((partial 0) f-rect) (up x0 y0)) (b0 (up x0 y0))) (* (((partial 1) f-rect) (up x0 y0)) (b1 (up x0 y0))))

In [21]:
;; This is Equation 3.6
(render *1)

"∂₀f-rect(up(x0, y0)) b0(up(x0, y0)) + ∂₁f-rect(up(x0, y0)) b1(up(x0, y0))"

In [9]:
((v2 (literal-manifold-function 'f-rect R2-rect)) R2-rect-point)

(+ (* (((partial 0) f-rect) (up x0 y0)) (b↑0 (up x0 y0))) (* (((partial 1) f-rect) (up x0 y0)) (b↑1 (up x0 y0))))

In [10]:
;; This is Equation 3.6
(render *1)

"∂₀f-rect(up(x0, y0)) b↑0(up(x0, y0)) + ∂₁f-rect(up(x0, y0)) b↑1(up(x0, y0))"

In [11]:
;; recover the coordinate components of the vector field 
;; by applying the vector field to the coordinate chart
((v2 (chart R2-rect)) R2-rect-point)

(up (b↑0 (up x0 y0)) (b↑1 (up x0 y0)))

In [12]:
((chart R2-rect) R2-rect-point)

(up x0 y0)

the procedure chart returns the "chart" which takes a manifold point and gives the coordinates under the chart. Technically speaking, the "chart" is not a manifold function. 

In [14]:
;; a signiture for functions that map an up structure of two reals to two reals.
;; (def R2->R2 '(-> (UP Real Real) (UP Real Real)))

#'chp3/R2->R2

In [18]:
;; (def f-R2->R2 (literal-manifold-function 'f-R2->R2 R2->R2))

#'chp3/f-R2->R2

In [22]:
(clojure.repl/source coordinatize)

(defn coordinatize
  [v coordinate-system]
  (let [coordinatized-v (fn [f]
                          (fn [x]
                            (let [b (f/compose (v (m/chart coordinate-system))
                                               (m/point coordinate-system))]
                              (g/* ((D f) x) (b x)))))]
    (o/make-operator coordinatized-v `(~'coordinatized ~v))))


nil

In [23]:
;; given a vector field and a coordinate system,
;; we can construct the coordinate representation of the vector field
(coordinatize v R2-rect)

(coordinatized (vector-field (up b0 b1)))

$\mathsf{v}(\mathsf{f})(\mathsf{m}) = v(f) (x)$

In [29]:
;; we can apply a coordinatized vector field to 
;; a function of coordinates (instead of a function of manifold point)
(((coordinatize v R2-rect) (literal-function 'f-rect R2->R))
 (up 'x0 'y0))

(+ (* (((partial 0) f-rect) (up x0 y0)) (b0 (up x0 y0))) (* (((partial 1) f-rect) (up x0 y0)) (b1 (up x0 y0))))

In [30]:
(render *1)

"∂₀f-rect(up(x0, y0)) b0(up(x0, y0)) + ∂₁f-rect(up(x0, y0)) b1(up(x0, y0))"

## 3.2 Coordinate-Basis Vector Fields
- for an n-dimensional manifold, any set of n linearly independent vector fields form a _basis_ in that any vector field can be expressed as a linear combination of the basis fields, with manifold-function coefficients. 
- The ith basis vector field is an operator that computes the directional derivative in the ith coordinate direction.
- $\mathsf{X}_i (\mathsf{f}) (\mathsf{m}) = \partial_i (\mathsf{f} \circ \chi^{-1}) (\chi(\mathsf{m})) $
- we can then express directional derivative in terms of basis vector fields
- $\mathsf{v}(\mathsf{f})(\mathsf{m}) 
= \sum_i \mathsf{X}_i (\mathsf{f}) (\mathsf{m})
  b^i (\chi(\mathsf{m}))$
- $\mathsf{v}(\mathsf{f})(\mathsf{m}) 
= \mathsf{X} (\mathsf{f}) (\mathsf{m})
  b (\chi(\mathsf{m}))$
- note that $b (\chi(\mathsf{m}))$ can be viewed as a coordinate tuple (coefficients) of the vector field under the basis vector filed $\mathsf{X}$
- the basis vector field is often written as $\mathsf{X}_i = \partial / \partial \mathsf{x}^i$

In [24]:
(coordinate-system->vector-basis R2-rect)

(down d:dx0 d:dx1)

In [25]:
(def d:dx (first (coordinate-system->vector-basis R2-rect)))
(def d:dy (second (coordinate-system->vector-basis R2-rect)))

#'chp3/d:dy

In [30]:
;; d:dx is a basis vector filed
;; (square r) is a coordinate function (a manifold function)
;; apply d:dx to (square r) gives another manifold function
;; in this way, we can construct more general manifold functions

(let-coordinates
 [[r theta] R2-polar]
 ((d:dx (square r)) R2-rect-point))

(+ (* (sqrt (+ (* x0 x0) (expt y0 2))) (/ 1 (* (sqrt (+ (* x0 x0) (expt y0 2))) 2)) (+ x0 x0)) (* (sqrt (+ (* x0 x0) (expt y0 2))) (/ 1 (* (sqrt (+ (* x0 x0) (expt y0 2))) 2)) (+ x0 x0)))

In [27]:
(render *1)

"2 x0"

In [28]:
(let-coordinates
 [[x y] R2-rect
  [r theta] R2-polar]
 (((+ d:dx (* 2 d:dy)) (+ (square r) (* 3 x))) R2-rect-point))

(+ (* (sqrt (+ (* x0 x0) (expt y0 2))) (/ 1 (* (sqrt (+ (* x0 x0) (expt y0 2))) 2)) (+ x0 x0)) (* (sqrt (+ (* x0 x0) (expt y0 2))) (/ 1 (* (sqrt (+ (* x0 x0) (expt y0 2))) 2)) (+ x0 x0)) 3 (* 2 (+ (* (sqrt (+ (expt x0 2) (* y0 y0))) (/ 1 (* (sqrt (+ (expt x0 2) (* y0 y0))) 2)) (+ y0 y0)) (* (sqrt (+ (expt x0 2) (* y0 y0))) (/ 1 (* (sqrt (+ (expt x0 2) (* y0 y0))) 2)) (+ y0 y0)))))

In [29]:
(render *1)

"2 x0 + 4 y0 + 3"

### Coordinate Transformations
- the basis elements transform _covariantly_ 
- the coefficients of a vector in terms of a basis transform _contravariantly_

## 3.3. Integral Curves
Notation explained:
- $\mathsf{v}$: a vector field on the manifold $\mathsf{M}$
- $\gamma_{\mathsf{m}}^{\mathsf{v}}: \mathsf{R} \rightarrow \mathsf{M}$ __integral curve__ is a parametric path on the manifold satisfying
    - $D(\mathsf{f} \circ \gamma_{\mathsf{m}}^{\mathsf{v}}) (t) = \mathsf{v}(\mathsf{f})(\gamma_{\mathsf{m}}^{\mathsf{v}}(t)) = (\mathsf{v}(\mathsf{f}) \circ \gamma_{\mathsf{m}}^{\mathsf{v}}) (t)$
    - $\gamma_{\mathsf{m}}^{\mathsf{v}}(0) = \mathsf{m}$
- $\phi_t^{\mathsf{v}}(\mathsf{m}) = \gamma_{\mathsf{m}}^{\mathsf{v}} (t)$



In [46]:
;; a convenient way to define a vector field
(let-coordinates
 [[x y] R2-rect]
 (def circular (- (* d:dy x) (* d:dx y))))

#'chp3/circular

In [47]:
circular

(- (* d:dy #function[clojure.core/comp/fn--5807]) (* d:dx #function[clojure.core/comp/fn--5807]))

In [48]:
;; exponentiate the vector field, 
;; and generate an evolution in a circle around the origin starting at (1, 0)
(take 6 
      (seq
       (((exp (* 't circular)) (chart R2-rect))
        ((point R2-rect) (up 1 0)))))

((up 1 0) (up 0 t) (up (* 1/2 t t -1) 0) (up 0 (* 1/6 t t -1 t)) (up (* 1/24 t t -1 t t -1) 0) (up 0 (* 1/120 t t -1 t t -1 t)))

In [49]:
(render *1)

"up(1, 0)(up(0, t), up(-1/2 t², 0), up(0, -1/6 t³), up(1/24 t⁴, 0), up(0, 1/120 t⁵))"

In [61]:
((((evolution 6) 'delta-t circular) (chart R2-rect))
 ((point R2-rect) (up 1 0)))

(up (+ 1 (* 1/2 delta-t delta-t -1) (* 1/24 delta-t delta-t -1 delta-t delta-t -1) (* 1/720 delta-t delta-t -1 delta-t delta-t -1 delta-t delta-t -1)) (+ delta-t (* 1/6 delta-t delta-t -1 delta-t) (* 1/120 delta-t delta-t -1 delta-t delta-t -1 delta-t)))

In [62]:
(render *1)

"up(-1/720 delta-t⁶ + 1/24 delta-t⁴ -1/2 delta-t² + 1, 1/120 delta-t⁵ -1/6 delta-t³ + delta-t)"

### Ex. 3.1: State Derivatives

define manifold $R^5$ and the rectangular coordinate system

In [35]:
(def R5 (make-manifold Rn 5))
(def R5-rect (coordinate-system-at :rectangular :origin R5))

#'chp3/R5

get basis vector fields

In [39]:
(coordinate-system->vector-basis R5-rect)

(down d:dx0 d:dx1 d:dx2 d:dx3 d:dx4)

#'chp3/R5-rect-vector-basis

In [79]:
(def d:dx0 (nth (coordinate-system->vector-basis R5-rect) 0))
(def d:dx1 (nth (coordinate-system->vector-basis R5-rect) 1))
(def d:dx2 (nth (coordinate-system->vector-basis R5-rect) 2))
(def d:dx3 (nth (coordinate-system->vector-basis R5-rect) 3))
(def d:dx4 (nth (coordinate-system->vector-basis R5-rect) 4))

#'chp3/d:dx4

In [80]:
(def A_x (literal-function 'A_x R2->R))
(def A_y (literal-function 'A_y R2->R))

#'chp3/A_y

In [81]:
(let-coordinates
 [[t x y v_x v_y] R5-rect]
 (def R5-vector-field
     (+ d:dx0
        (* d:dx1 v_x)
        (* d:dx2 v_y)
        (* d:dx3 (compose A_x (up x y)))
        (* d:dx4 (compose A_y (up x y))))))

#'chp3/R5-vector-field

In [82]:
(take 4
      (seq
       (((exp (* 't R5-vector-field)) (chart R5-rect))
        ((point R5-rect) (up 1 1 2 0 0)))))

((up 1 1 2 0 0) (up t 0 0 (* t (A_x (up 1 2))) (* t (A_y (up 1 2)))) (up 0 (* 1/2 t (A_x (up 1 2)) t) (* 1/2 t (A_y (up 1 2)) t) 0 0) (up 0 0 0 (* 1/6 t (+ (* (A_x (up 1 2)) t t (((partial 0) A_x) (up 1 2))) (* (A_y (up 1 2)) t t (((partial 1) A_x) (up 1 2))))) (* 1/6 t (+ (* (A_x (up 1 2)) t t (((partial 0) A_y) (up 1 2))) (* (A_y (up 1 2)) t t (((partial 1) A_y) (up 1 2)))))))

In [83]:
(render *1)

"up(1, 1, 2, 0, 0)(up(t, 0, 0, t A_x(up(1, 2)), t A_y(up(1, 2))), up(0, 1/2 t² A_x(up(1, 2)), 1/2 t² A_y(up(1, 2)), 0, 0), up(0, 0, 0, 1/6 t³ A_x(up(1, 2)) ∂₀A_x(up(1, 2)) + 1/6 t³ A_y(up(1, 2)) ∂₁A_x(up(1, 2)), 1/6 t³ A_x(up(1, 2)) ∂₀A_y(up(1, 2)) + 1/6 t³ A_y(up(1, 2)) ∂₁A_y(up(1, 2))))"

We can also get the vector field by inner product, so that we don't have to name those basis vector fields

In [86]:
(def R5-rect-vector-basis (coordinate-system->vector-basis R5-rect))

#'chp3/R5-rect-vector-basis

In [87]:
(let-coordinates
 [[t x y v_x v_y] R5-rect]
 (def R5-vector-field-coefs
     (up 1
         v_x
         v_y
         (compose A_x (up x y))
         (compose A_y (up x y))))
 (def R5-vector-field2
     (* R5-rect-vector-basis R5-vector-field-coefs)))

#'chp3/R5-vector-field2

In [88]:
(take 4
      (seq
       (((exp (* 't R5-vector-field2)) (chart R5-rect))
        ((point R5-rect) (up 1 1 2 0 0)))))

((up 1 1 2 0 0) (up t 0 0 (* t (A_x (up 1 2))) (* t (A_y (up 1 2)))) (up 0 (* 1/2 t (A_x (up 1 2)) t) (* 1/2 t (A_y (up 1 2)) t) 0 0) (up 0 0 0 (* 1/6 t (+ (* (A_x (up 1 2)) t t (((partial 0) A_x) (up 1 2))) (* (A_y (up 1 2)) t t (((partial 1) A_x) (up 1 2))))) (* 1/6 t (+ (* (A_x (up 1 2)) t t (((partial 0) A_y) (up 1 2))) (* (A_y (up 1 2)) t t (((partial 1) A_y) (up 1 2)))))))

In [89]:
(render *1)

"up(1, 1, 2, 0, 0)(up(t, 0, 0, t A_x(up(1, 2)), t A_y(up(1, 2))), up(0, 1/2 t² A_x(up(1, 2)), 1/2 t² A_y(up(1, 2)), 0, 0), up(0, 0, 0, 1/6 t³ A_x(up(1, 2)) ∂₀A_x(up(1, 2)) + 1/6 t³ A_y(up(1, 2)) ∂₁A_x(up(1, 2)), 1/6 t³ A_x(up(1, 2)) ∂₀A_y(up(1, 2)) + 1/6 t³ A_y(up(1, 2)) ∂₁A_y(up(1, 2))))"

### 3.5 Coordinate-Basis One-Form Fields

In [91]:
(def R2->R '(-> (UP Real Real) Real))

#'chp3/R2->R

In [92]:
(def omega 
    (components->oneform-field
     (down (literal-function 'a_0 R2->R)
           (literal-function 'a_1 R2->R))
     R2-rect))

#'chp3/omega

In [95]:
((omega d:dx) R2-rect-point)

(a_0 (up x0 y0))

In [96]:
((omega (down d:dx d:dy)) R2-rect-point)

(down (a_0 (up x0 y0)) (a_1 (up x0 y0)))

In [97]:
(def omega2 (literal-oneform-field 'a R2-rect))

#'chp3/omega2

In [101]:
(((d (literal-manifold-function 'f-rect R2-rect))
  (coordinate-system->vector-basis R2-rect))
 R2-rect-point)

(down (((partial 0) f-rect) (up x0 y0)) (((partial 1) f-rect) (up x0 y0)))

In [103]:
(((d (literal-manifold-function 'f-polar R2-polar))
  (coordinate-system->vector-basis R2-rect))
 ((point R2-polar) (up 'r 'theta)))

(down (+ (* (((partial 0) f-polar) (up (sqrt (+ (* r (cos theta) r (cos theta)) (expt (* r (sin theta)) 2))) (atan (* r (sin theta)) (* r (cos theta))))) (/ 1 (* (sqrt (+ (* r (cos theta) r (cos theta)) (expt (* r (sin theta)) 2))) 2)) (+ (* r (cos theta)) (* r (cos theta)))) (* (((partial 1) f-polar) (up (sqrt (+ (* r (cos theta) r (cos theta)) (expt (* r (sin theta)) 2))) (atan (* r (sin theta)) (* r (cos theta))))) (/ (- (* r (sin theta))) (+ (expt (* r (cos theta)) 2) (expt (* r (sin theta)) 2))))) (+ (* (((partial 0) f-polar) (up (sqrt (+ (expt (* r (cos theta)) 2) (* r (sin theta) r (sin theta)))) (atan (* r (sin theta)) (* r (cos theta))))) (/ 1 (* (sqrt (+ (expt (* r (cos theta)) 2) (* r (sin theta) r (sin theta)))) 2)) (+ (* r (sin theta)) (* r (sin theta)))) (* (((partial 1) f-polar) (up (sqrt (+ (expt (* r (cos theta)) 2) (* r (sin theta) r (sin theta)))) (atan (* r (sin theta)) (* r (cos theta))))) (/ (* r (cos theta)) (+ (expt (* r (cos theta)) 2) (expt (* r (sin theta)) 2

In [104]:
(render *1)

"down((r cos(theta) ∂₀f-polar(up(r, theta)) - sin(theta) ∂₁f-polar(up(r, theta))) / r, (r sin(theta) ∂₀f-polar(up(r, theta)) + cos(theta) ∂₁f-polar(up(r, theta))) / r)"

duality of the coordinate-basis vector fields and the coordinate-basis one-form fields

In [114]:
(let-coordinates
 [[x y] R2-rect]
 ((dx d:dy) R2-rect-point))

0

In [115]:
(let-coordinates
 [[x y] R2-rect]
 ((dx d:dx) R2-rect-point))

1

use coordinate-basis one-form fields to find the coefficients of vector fields in the coresponding coordinate vector-field basis. Here we basically get the coefficient functions evaluated at an arbitrary point.

In [116]:
(let-coordinates
 [[x y] R2-rect]
 ((dx circular) R2-rect-point))

(- y0)

In [117]:
(let-coordinates
 [[x y] R2-rect]
 ((dy circular) R2-rect-point))

x0

In [119]:
;; remember how the circular function is defined
circular

(- (* d:dy #function[clojure.core/comp/fn--5807]) (* d:dx #function[clojure.core/comp/fn--5807]))

find coefficients on the polar vector basis.

In [120]:
(let-coordinates
 [[r theta] R2-polar]
 ((dr circular) R2-rect-point))

(- (* x0 (/ 1 (* (sqrt (+ (expt x0 2) (* y0 y0))) 2)) (+ y0 y0)) (* y0 (/ 1 (* (sqrt (+ (* x0 x0) (expt y0 2))) 2)) (+ x0 x0)))

In [121]:
(render *1)

"0"

In [122]:
(let-coordinates
 [[r theta] R2-polar]
 ((dtheta circular) R2-rect-point))

(- (* x0 (/ x0 (+ (expt x0 2) (expt y0 2)))) (* y0 (/ (- y0) (+ (expt x0 2) (expt y0 2)))))

In [123]:
(render *1)

"1"

In [126]:
(def f (literal-manifold-function 'f-rect R2-rect))
(let-coordinates
 [[r theta] R2-polar]
 (((- circular d:dtheta) f) R2-rect-point))

(- (- (* x0 (((partial 1) f-rect) (up x0 y0))) (* y0 (((partial 0) f-rect) (up x0 y0)))) (+ (* (((partial 0) f-rect) (up (* (sqrt (+ (expt x0 2) (expt y0 2))) (cos (atan y0 x0))) (* (sqrt (+ (expt x0 2) (expt y0 2))) (sin (atan y0 x0))))) (sqrt (+ (expt x0 2) (expt y0 2))) (- (sin (atan y0 x0)))) (* (((partial 1) f-rect) (up (* (sqrt (+ (expt x0 2) (expt y0 2))) (cos (atan y0 x0))) (* (sqrt (+ (expt x0 2) (expt y0 2))) (sin (atan y0 x0))))) (sqrt (+ (expt x0 2) (expt y0 2))) (cos (atan y0 x0)))))

In [127]:
(render *1)

"0"

### Coordinate Transformations

In [129]:
(def omega (literal-oneform-field 'a R2-rect))
(def v (literal-vector-field 'b R2-rect))

#'chp3/v

In [130]:
((omega v) R2-rect-point)

(+ (* (a_0 (up x0 y0)) (b↑0 (up x0 y0))) (* (a_1 (up x0 y0)) (b↑1 (up x0 y0))))

### Ex3.2: Verification
Verify that the coefficients of a one-form field transform as described in equation (3.56). You should use equation (3.44) in your derivation.

__Solution__: We use the transform between rectangular coordinate system and polar coordinate system in $R_2$ to verigy equation (3.56)

In [177]:
;; this is essentially Eqn. 3.44 for system 1
(let-coordinates
 [[x y] R2-rect]
 (def a1 ((omega (down d:dx d:dy)) R2-rect-point)))

#'chp3/a1

In [179]:
(render a1)

"down(a₀(up(x0, y0)), a₁(up(x0, y0)))"

In [178]:
;; this is essentially Eqn. 3.44 for system 2
(let-coordinates
 [[r theta] R2-polar]
 (def a2 ((omega (down d:dr d:dtheta)) R2-rect-point)))

#'chp3/a2

In [162]:
(render a2)

"down((x0 a₀(up(x0, y0)) + y0 a₁(up(x0, y0))) / sqrt(x0² + y0²), x0 a₁(up(x0, y0)) - y0 a₀(up(x0, y0)))"

We get the jacobian evaluated at an arbitrary point $D(\chi_1 \circ (\chi_2)^{-1}) \circ (\chi_2) (\mathsf{m})$

In [164]:
(def jacobian
    ((compose (D (compose (chart R2-rect) (point R2-polar))) 
              (chart R2-polar))
     R2-rect-point))

#'chp3/jacobian

In [180]:
(render jacobian)

"down(up(x0 / sqrt(x0² + y0²), y0 / sqrt(x0² + y0²)), up(- y0, x0))"

In [168]:
;; this is essentially verifying Eqn. (3.55)
(- a2 (* a1 jacobian))

(down 0 0)

In [175]:
;; this is essentially verifying Eqn. (3.56)
(- a1 (* a2 (/ 1 jacobian)))

(down (- (a_0 (up x0 y0)) (+ (* (+ (* (a_0 (up x0 y0)) (cos (atan y0 x0))) (* (a_1 (up x0 y0)) (sin (atan y0 x0)))) (/ (* (sqrt (+ (expt x0 2) (expt y0 2))) (cos (atan y0 x0))) (+ (* (cos (atan y0 x0)) (sqrt (+ (expt x0 2) (expt y0 2))) (cos (atan y0 x0))) (* (sqrt (+ (expt x0 2) (expt y0 2))) (- (sin (atan y0 x0))) (- (sin (atan y0 x0))))))) (* (+ (* (a_0 (up x0 y0)) (sqrt (+ (expt x0 2) (expt y0 2))) (- (sin (atan y0 x0)))) (* (a_1 (up x0 y0)) (sqrt (+ (expt x0 2) (expt y0 2))) (cos (atan y0 x0)))) (/ (- (sin (atan y0 x0))) (+ (* (cos (atan y0 x0)) (sqrt (+ (expt x0 2) (expt y0 2))) (cos (atan y0 x0))) (* (sqrt (+ (expt x0 2) (expt y0 2))) (- (sin (atan y0 x0))) (- (sin (atan y0 x0))))))))) (- (a_1 (up x0 y0)) (+ (* (+ (* (a_0 (up x0 y0)) (cos (atan y0 x0))) (* (a_1 (up x0 y0)) (sin (atan y0 x0)))) (/ (- (* (sqrt (+ (expt x0 2) (expt y0 2))) (- (sin (atan y0 x0))))) (+ (* (cos (atan y0 x0)) (sqrt (+ (expt x0 2) (expt y0 2))) (cos (atan y0 x0))) (* (sqrt (+ (expt x0 2) (expt y0 2))) (

In [176]:
(render *1)

"down(0, 0)"

### Ex 3.3 Hill Climbing
- The topography of a region on the Earth can be specified by a manifold function $\mathsf{h}$ that gives the altitude at each point on the manifold. 
- Let $\mathsf{v}$ be a vector field on the manifold, specifying a direction and rate of walking at every point on the manifold

__a.__ Form an expression that gives the power that must be expended to follow the vector field at each point. 

__Solution:__ for a mass $m$ and the standard gravity $g$ the manifold function $mg\cdot\mathsf{v} (\mathsf{h})$ gives the required power at each point. This is because $\mathsf{v} (\mathsf{h})$ gives the rate of change of height as one moves in the way described by the vector field (see Section 3.4.)

__b.__ Write this as a computational expression.

__Solution:__

In [182]:
(point S2-Riemann)

#object[sicmutils.calculus.manifold$__GT_Stereographic$ctor$reify__20810 0x2ca95893 "sicmutils.calculus.manifold$__GT_Stereographic$ctor$reify__20810@2ca95893"]

In [183]:
;; a literal vector field
(def v
    (literal-vector-field 'b S2-Riemann))

#'chp3/v

In [184]:
(def h
    (literal-manifold-function 'h S2-Riemann))

#'chp3/h

In [185]:
(def S2-Riemann-point ((point S2-Riemann) (up 'x0 'y0)))

#'chp3/S2-Riemann-point

In [189]:
(* 'm 'g ((v h) S2-Riemann-point))

(* m g (+ (* (((partial 0) h) (up x0 y0)) (b↑0 (up x0 y0))) (* (((partial 1) h) (up x0 y0)) (b↑1 (up x0 y0)))))

In [190]:
(render *1)

"g m b↑0(up(x0, y0)) ∂₀h(up(x0, y0)) + g m b↑1(up(x0, y0)) ∂₁h(up(x0, y0))"